#Task 1: Import Libraries

In [1]:
!pip install transformers torch accelerate

In [2]:
import torch
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

#Task 2: Load Lightweight model


In [3]:
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

print("Model Loaded Successfully ✅")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

Model Loaded Successfully ✅


#Task 3: Initialise FAQ knowledge Base

In [4]:
faq_dict = {
    "return policy": "Our return policy allows returns within 30 days with a valid receipt.",
    "refund": "Refunds are processed within 5-7 business days after approval.",
    "shipping": "Standard shipping takes 3-5 business days.",
    "cancel order": "To cancel your order, please provide your order ID.",
    "contact": "You can contact our support team at support@company.com."
}


#Task 4: FAQ Detection Function


In [5]:
def check_faq(user_input):
    user_input = user_input.lower()
    for key in faq_dict:
        if key in user_input:
            return faq_dict[key]
    return None

In [6]:
#Conversation memory
conversation = [
    {"role": "system", "content": "You are a professional and polite customer service assistant. Always reply clearly and briefly."}
]

#Task 5: AI Response Generator(With Memory)

In [7]:
def generate_ai_response(user_input):

    global conversation

    conversation.append({"role": "user", "content": user_input})

    # Apply chat template
    prompt = tokenizer.apply_chat_template(
        conversation,
        tokenize=False,
        add_generation_prompt=True
    )

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=80,
        temperature=0.7,
        do_sample=True,
        eos_token_id=tokenizer.eos_token_id
    )

    reply = tokenizer.decode(outputs[0], skip_special_tokens=True)

    reply = reply.split("assistant")[-1].strip()

    conversation.append({"role": "assistant", "content": reply})

    return reply

#Task 6: Initialise Chatbot Conversation

In [8]:
print("======================================")
print("🤖 Welcome to Smart Customer Service Chatbot")
print("Type 'exit' to end conversation")
print("======================================\n")

🤖 Welcome to Smart Customer Service Chatbot
Type 'exit' to end conversation



#Task 7: Testing section



In [9]:
test_questions = [
    "What is your return policy?",
    "I want a refund",
    "My order is delayed"
]

print("Running Test Cases...\n")

for question in test_questions:
    print("You:", question)

    faq_response = check_faq(question)

    if faq_response:
        print("Chatbot (FAQ):", faq_response)
    else:
        print("Chatbot (AI):", generate_ai_response(question))

    print("-" * 50)

Running Test Cases...

You: What is your return policy?
Chatbot (FAQ): Our return policy allows returns within 30 days with a valid receipt.
--------------------------------------------------
You: I want a refund
Chatbot (FAQ): Refunds are processed within 5-7 business days after approval.
--------------------------------------------------
You: My order is delayed
Chatbot (AI): |>
I'm sorry to hear about your order being delayed. Please let us know what time your package was shipped, and if there are any specific details we can provide about the delay. This information will help us understand the situation better and ensure that your package arrives as soon as possible. Please let us know if there are any updates on the delivery status as well. We'll do our
--------------------------------------------------


#Interactive Chat

In [10]:
while True:
    user_input = input("You: ")

    if user_input.lower() == "exit":
        print("Chatbot: Thank you for contacting us. Have a great day! 😊")
        break

    faq_response = check_faq(user_input)

    if faq_response:
        print("Chatbot (FAQ):", faq_response)
    else:
        reply = generate_ai_response(user_input)
        print("Chatbot (AI):", reply)

You: hi
Chatbot (AI): |>
Sure, please provide me with the following details:
- The time your package was shipped
- Any specific details on the delay, such as when the package was shipped, when it was initially scheduled to arrive, and any delays or issues related to that timeframe
- If there have been any updates on the delivery status, such as when the package was expected to arrive or
You: thanks
Chatbot (AI): |>
I'll be happy to help you. Please provide me with the following information:

- The time of the package shipping, and the expected delivery date or window
- The name of the company shipping the package, and any specific details about their delivery process
- Any updates on the delivery status, such as when the package was expected to arrive or when the package was delivered to
You: exit
Chatbot: Thank you for contacting us. Have a great day! 😊
